# Uczenie maszynowe

### Modelowanie

Przykład: zbierasz pieniądze na uruchomienie własnego serwisu społecznościowego. 

Model biznesowy przyjmuje dane wejściowe, takie jak "liczba użytkowników", "przychód z reklam na użytkownika" i "liczba pracowników", a następnie zwraca roczne zyski generowane przez serwis w ciągu kilku kolejnych lat istnienia.

### Czym jest uczenie maszynowe

Uczenie maszynowe jest procesem tworzenia i używania modeli, które uczą się na odstawie danych.

Zwykle naszym celem będzie zastosowanie dostępnych danych do utworzenia modeli, które mogą być użyte do przewidzenia nowych danych, np:
- przewidzenia, czy wiadomość e-mail jest spamem
- przewidzenia, że transakcja wykonana kartą kredytową jest oszustwem
- przewidzenia, którą reklamę kliknie klient

Model nadzorowany - model uczy się na zbiorze danych zawierającym etykiety z poprawnymi odpowiedziami

Model nienadzorowany - model nie ma dostępu do etykiet

Model uczenia częściowo nadzorowanego - tylko niektóre elementy zbioru danych są opatrzone etykietami

Model stale uczący sie - model musi dostosować się do stałego dopływu nowych danych

Model uczenia przez wzmacnianie - model po serii przewisywań otrzymuje informację zwrotną na temat ich skuteczności

### Nadmierne i zbyt małe dopasowanie

Nadmierne dopasowanie - wygenerowanie modelu, który doskonale sprawdza się na treningowym zbiorze danych, ale generuje słabe wyniki w przypadku nowych danych

Zbyt słabe dopasowanie - wygenerowanie modelu, który daje kiepskie efekty nawet na teningowym zbiorze danych

Np. dopasowanie wielomianów

In [1]:
import random
from typing import TypeVar, List, Tuple
X = TypeVar('X')  # generyczny typ do reprezentowania punktów danych

def split_data(data: List[X], prob: float) -> Tuple[List[X], List[X]]:
    """Podziel dane na zbiór treningowy i testowy."""
    data = data[:]                    # Zrób tzw. płytką kopię,
    random.shuffle(data)              # ponieważ funkcja shuffle modyfikuje listę.
    cut = int(len(data) * prob)       # Użyj parametru prob, aby znaleźć punkt podziału,
    return data[:cut], data[cut:]     # i rozdziel listę w tym punkcie.

data = [n for n in range(1000)]
train, test = split_data(data, 0.75)

In [2]:
# Proporcje powinny się zgadzać
# len(train) == 750
len(train)

750

In [3]:
# len(test) == 250
len(test)

250

In [4]:
# A oryginalne dane powinny być zachowane (w odpowiednim porządku)
assert sorted(train + test) == data

Np. Dysponujemy zmiennymi wejściowymi i zmiennymi wyjściowymi, które są sparowane. W takim przypadku należy upewnić się, że w zbiorze tranigowym i testowym znalazły sie odpowiadające sobie zmienne

In [5]:
Y = TypeVar('Y')  # generyczny typ do reprezentacji danych wyjściowych

def train_test_split(xs: List[X],
                     ys: List[Y],
                     test_pct: float) -> Tuple[List[X], List[X], List[Y], List[Y]]:
    # Tworzy indeksy i rozdziela je.
    idxs = [i for i in range(len(xs))]
    train_idxs, test_idxs = split_data(idxs, 1 - test_pct)

    return ([xs[i] for i in train_idxs],  # x_train
            [xs[i] for i in test_idxs],   # x_test
            [ys[i] for i in train_idxs],  # y_train
            [ys[i] for i in test_idxs])   # y_test

In [6]:
xs = [x for x in range(1000)]  # xs to wartości 1 … 1000
ys = [2 * x for x in xs]       # każda wartość y_i to podwojona wartość x_i
x_train, x_test, y_train, y_test = train_test_split(xs, ys, 0.25)

In [7]:
# Sprawdzamy, czy proporcje się zgadzają
#len(x_train) == len(y_train) == 750

len(x_train) == len(y_train) == 750

True

In [8]:
len(x_test) == len(y_test) == 250

True

In [9]:
# Sprawdzamy, czy odpowiednie punkty w danych są ze sobą sparowane 
assert all(y == 2 * x for x, y in zip(x_train, y_train))
assert all(y == 2 * x for x, y in zip(x_test, y_test))

### Poprawność

Np. model generujący wartości binarne określające to, czy dany e-mail jest spamem. Dysponujemy zbiorem danych oznaczonych etykietami, możemy stwierdzić przynależność każdego wyniku przewidywania zwróconego przez model do jednej z czterech kategorii:
- wynik prawdziwie pozytywny: "ta wiadomość jest spamem i poprawnie to przewidzieliśmy"
- wynik fałszywie pozytywny (błąd typu 1): "ta wiadomość nie jest spamem, ale według modelu miała być spamem"
- wynik fałszywie nagatywny (błąd typu 2): "ta wiadomość jest spamem, ale według modelu miała nim nie być"
- wynik prawdziwie negatywny: "ta wiadomość nie jest spamem i model przewidział to poprawnie"

Inny przykład: występowanie białaczki. Imię Luke nadawane jest ok 5 na 1000 nowo narodzonych dzieci. Na białaczkę w przeciągu życia choruje ok 1,4% populacji (14 na 1000 osób). Załóżmy, że te dwa czynniki są niezależne i przyjmijmy tezę, że "Luke choruje na białaczkę", to w przypadku przeprowadzenia testu na próbie miliona osób:

                        białaczka         Brak białaczki       Suma
    imię Luke             70                 4930             5000
    imię inne niż Luke   13930              981070            995000
    Suma                 14000              986000            1000000

In [10]:
# Dokładność
def accuracy(tp: int, fp: int, fn: int, tn: int) -> float:
    correct = tp + tn
    total = tp + fp + fn + tn
    return correct / total

In [11]:
accuracy(70, 4930, 13930, 981070)

0.98114

In [12]:
# Precyzja
def precision(tp: int, fp: int, fn: int, tn: int) -> float:
    return tp / (tp + fp)

In [13]:
precision(70, 4930, 13930, 981070)

0.014

In [14]:
# współczynnik przewydywania
def recall(tp: int, fp: int, fn: int, tn: int) -> float:
    return tp / (tp + fn)

In [15]:
recall(70, 4930, 13930, 981070)

0.005

Współczynniki przyjmują niskie wartości, co świadczy o tym, że test jest kiepskim modelem.

In [16]:
# Czasem precyzję i współczynnik przewidywania łączy się w miarę F1 - jest to średnia harmoniczna
def f1_score(tp: int, fp: int, fn: int, tn: int) -> float:
    p = precision(tp, fp, fn, tn)
    r = recall(tp, fp, fn, tn)

    return 2 * p * r / (p + r)